In [1]:
import os
from typing import List

import numpy as np
import pandas as pd
import seaborn as sns
from comet_ml import Experiment
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow_hub as hub

In [2]:
train_df = pd.read_csv('../data/tweets_50_shuffled_train.csv')
test_df = pd.read_csv('../data/tweets_50_shuffled_test.csv')
X_train = train_df['text_tokenized']
y_train = train_df['label']
X_test = test_df['text_tokenized']
y_test = test_df['label']

class_names = y_train.unique()

In [3]:
batch_size = 128
tfhub_embedding_models = ["https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1", "https://tfhub.dev/google/nnlm-en-dim128/2"]
activation = 'relu'
kernel_initializer = 'he_normal'
l2_lambda = 1e-3
dropout=0.5
filters = 64
kernel_sizes = [1, 2, 3]
n_convs_parallel = len(kernel_sizes)
padding='same'
pool_size = 2
strides = 1
epochs = 200
optimizer = 'nadam'

for embed_model in tfhub_embedding_models:
    def make_model():
        regularizers = keras.regularizers.l2(l2=l2_lambda)
        model = keras.Sequential([
            hub.KerasLayer(embed_model, dtype=tf.string, input_shape=[],),
            keras.layers.Dense(128, activation=activation, kernel_initializer=kernel_initializer, kernel_regularizer=regularizers),
            keras.layers.Dense(len(class_names), activation="softmax")
        ])
        model.summary()

        model.compile(
            loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["acc"]
        )
        return model

    project_name = 'nlp_split'
    experiment = Experiment(
        project_name=project_name,
        auto_param_logging=True,
        # auto_histogram_weight_logging=True,
        auto_histogram_gradient_logging=True,
        auto_histogram_activation_logging=True,
        api_key="HeH9EtfDC2KUlCOjeQaU1CuOM",
        workspace="henrystoll",
    )
    params = {
        'batch_size': batch_size,
        'bert_model_name': embed_model,
        'filters': filters,
        'kernel_sizes': kernel_sizes,
        'pool_size': pool_size,
        'padding': padding,
        'strides': strides,
        'n_convs_parallel': n_convs_parallel,
        'activation': activation,
        'kernel_initializer': kernel_initializer,
        'l2_lambda': l2_lambda,
        'dropout': dropout,
        'optimizer': optimizer,
        'epochs': epochs,
    }

    experiment.log_parameters(params)

    model = make_model()
    # keras.utils.plot_model(model, "model.png", show_shapes=True)
    # experiment.log_asset("model.png")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=30,
        verbose=1,
        restore_best_weights=True)

    with experiment.train():
        history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(X_test, y_test),
                            verbose=1,
                            callbacks=[early_stopping])

    with experiment.test():
        loss, accuracy = model.evaluate(X_test, y_test)
        print('acc : {:.3f}'.format(accuracy))
        metrics = {
            'loss': loss,
            'accuracy': accuracy
        }
        experiment.log_metrics(metrics)

    y_predicted = model.predict(X_test)
    y_predicted = y_predicted.argmax(axis=1)
    experiment.log_confusion_matrix(y_test.to_numpy(), y_predicted)

    experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/henrystoll/nlp-split/4f1d035fe2c148d59521d907d51ca5fb

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 128)               2688      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 403,095
Trainable params: 3,075
Non-trainable params: 400,020
_________________________________________________________________
Epoch 1/200
59/59 [==============================] - 6s 76ms/step - loss: 1.9700 - acc: 0.4326 - val_loss: 1.8170 - val_acc: 0.4313
Epoch 2/200
59/59 [==============================] - 4s 66ms/step - loss: 1.3057 - acc: 0.5388 - val_loss: 1.5130 - val_acc: 0.4281
Epoch 3/200
59/59 [==============================] - 4s 67ms/step - loss: 1.1699 - acc: 0.5833 - val_loss: 1.349

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/henrystoll/nlp-split/4f1d035fe2c148d59521d907d51ca5fb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_accuracy             : 0.6311770081520081
COMET INFO:     test_loss                 : 1.0446381568908691
COMET INFO:     train_acc [53]            : (0.43261075019836426, 0.7754140496253967)
COMET INFO:     train_batch_acc [318]     : (0.34375, 0.7791053652763367)
COMET INFO:     train_batch_loss [318]    : (0.7070590257644653, 3.8854668140411377)
COMET INFO:     train_epoch_duration [53] : (3.613564258965198, 5.891675059974659)
COMET INFO:     train_loss [53]           : (0.7094767093658447, 1.9700229167938232)
COMET INFO:     train_val_acc [53]        : (0.20934411883354187, 0.6311770081520081)
COMET INFO:     train_val_loss [53] 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 124,659,587
Trainable params: 16,899
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/200
59/59 [==============================] - 6s 84ms/step - loss: 1.2689 - acc: 0.4909 - val_loss: 1.1708 - val_acc: 0.5800
Epoch 2/200
59/59 [==============================] - 5s 78ms/step - loss: 1.0689 - acc: 0.6231 - val_loss: 1.7439 - val_acc: 0.2758
Epoch 3/200
59/59 [==============================] - 5s 77ms/step - loss: 0.9751 - acc: 0.6752 - val_

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/henrystoll/nlp-split/ab4c9ac6620b4a1b8b6cec33f2ba2536
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_accuracy             : 0.5799640417098999
COMET INFO:     test_loss                 : 1.1708489656448364
COMET INFO:     train_acc [31]            : (0.4909115433692932, 0.9114043116569519)
COMET INFO:     train_batch_acc [186]     : (0.3046875, 0.918154776096344)
COMET INFO:     train_batch_loss [186]    : (0.4044302999973297, 1.642451286315918)
COMET INFO:     train_epoch_duration [31] : (4.328057378006633, 7.4144913359778)
COMET INFO:     train_loss [31]           : (0.4118863046169281, 1.268898606300354)
COMET INFO:     train_val_acc [31]        : (0.2529200315475464, 0.5799640417098999)
COMET INFO:     train_val_loss [31]      